In [1]:
#import sys
#sys.path.append('/dss/dsshome1/0F/di93quv/Systems_biomedicine/acdc/')
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [16]:
#!pip install pandas numpy scikit-learn scipy seaborn matplotlib phenograph

In [8]:
import time
import pandas as pd
import numpy as np
from collections import Counter
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph

In [4]:
channels = ['CD45','CD45RA', 'CD19', 'CD11b', 'CD4', 'CD8', 'CD34',
           'CD20', 'CD33', 'CD123', 'CD38', 'CD90', 'CD3']

# Load the data
path = '/dss/dsshome1/0F/di93quv/Systems_biomedicine/acdc/data/BMMC_benchmark/'
df = pd.read_csv(path + 'BMMC_benchmark.csv.gz', sep=',', header=0, compression='gzip')
df = df[df.cell_type != 'NotGated']


table = pd.read_csv(path + 'BMMC_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

X0= np.arcsinh((df[channels].values - 1.0)/5.0)

In [6]:
# Define helper dictionaries
idx2ct = [key for idx, key in enumerate(table.index)]
idx2ct.append('unknown')

ct2idx = {key: idx for idx, key in enumerate(table.index)}
ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.to_numpy()).sum(axis=1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    else:
        y0[i] = ct2idx['unknown']

In [7]:
thres = 0.5

In [10]:
tic = time.perf_counter()  # Start the timer
phenograph.cluster(X0)
toc = time.perf_counter()  # Stop the timer
print(f"Elapsed time: {toc - tic:.2f} seconds")

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 50.53551173210144 seconds
Jaccard graph constructed in 22.817151308059692 seconds
Wrote graph to binary file in 1.556551456451416 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.878095
Louvain completed 21 runs in 116.6292769908905 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 192.89297580718994 seconds
Elapsed time: 192.90 seconds


In [12]:
result = []
score_final = []
process_time = []

X = X0
y_true = y0

# Start timing
tic = time.perf_counter()
mk_model = compute_marker_model(pd.DataFrame(X, columns=channels), table, 0.0)

# Compute posterior probabilities
score = get_score_mat(X, [], table, [], mk_model)
score = np.concatenate([score, 1.0 - score.max(axis=1)[:, np.newaxis]], axis=1)

# Get indices
ct_index = get_unique_index(X, score, table, thres)

# Running ACDC
tic = time.perf_counter()  # Start timing for ACDC
res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))])
landmark_label = np.array(landmark_label)

# End timing for ACDC
toc = time.perf_counter()

# Compute elapsed time
time0 = toc - tic
print(f"ACDC processing time: {time0:.2f} seconds")

Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 6.783504247665405 seconds
Jaccard graph constructed in 2.8109443187713623 seconds
Wrote graph to binary file in 0.30356287956237793 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.655157
Louvain completed 21 runs in 11.653422355651855 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 22.4973464012146 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 1.1239781379699707 seconds
Jaccard graph constructed in 2.373694896697998 seconds
Wrote graph to binary file in 0.15328764915466309 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.602655
After 3 runs, maximum modularity is Q = 0.60542
After 10 runs, maximum modularity is Q = 0.606848
After 15 runs, maximum modularity is Q = 0.608913
Louvain completed 35 runs in 10.749405145645142 second

In [13]:
score_final = []
process_time = []

for n_neighbor in [10, 20, 30]:
    # Start timing
    tic = time.perf_counter()
    
    # Perform classification
    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)
    
    # End timing
    toc = time.perf_counter()
    time1 = toc - tic  # Time for this iteration
    
    # Add total processing time
    process_time.append(time0 + time1)

    # Append the accuracy score
    score_final.append(accuracy_score(y_true, [ct2idx[c] for c in y_pred]))    

# Print the results
print("Processing times:", process_time)
print("Accuracy scores:", score_final)

Processing times: [254.2722597781103, 287.731193800224, 317.5095457902644]
Accuracy scores: [0.9280707548900877, 0.9228840202086926, 0.9193731880068994]


In [15]:
X.shape

(81747, 13)